# 1. 환경 설정

In [ ]:
!pip install ultralytics

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 896.9/896.9 kB 14.7 MB/s eta 0:00:00


In [ ]:
import os
from ultralytics import YOLO
import time

Creating new Ultralytics Settings v0.0.6 file ✅ 
View Ultralytics Settings with 'yolo settings' or at '/root/.config/Ultralytics/settings.json'
Update Settings with 'yolo settings key=value', i.e. 'yolo settings runs_dir=path/to/dir'. For help see https://docs.ultralytics.com/quickstart/#ultralytics-settings.


# 2. 경로 지정

In [ ]:
from google.colab import drive

# Google Drive 연결
drive.mount('/content/drive')

Mounted at /content/drive


In [ ]:
# 모델 로드
model_path = "/content/drive/MyDrive/nipa_google/model/yolov8n_640/weights/best.pt"

# 데이터 경로 설정
root_dir = "/content/drive/MyDrive/nipa_google/data/test/yolov8"
images_dir = os.path.join(root_dir, "images")  # 테스트 이미지 경로
labels_dir = os.path.join(root_dir, "labels")  # 테스트 라벨 경로

In [ ]:
import os

# 클래스 정보 설정
class_names = ["0", "1", "2"]

# YAML 파일 경로
yaml_path = os.path.join(root_dir, "dataset.yaml")

# YAML 파일 생성
yaml_content = f"""
path: {root_dir}  # 데이터셋 루트 경로
train:  # 더미 경로
val:   # 더미 경로
test: images  # 테스트 이미지 상대 경로
nc: {len(class_names)}  # 클래스 수
names: {class_names}  # 클래스 이름
"""

# 파일 저장
with open(yaml_path, "w") as f:
    f.write(yaml_content)

print(f"YAML 파일이 생성되었습니다: {yaml_path}")
print("내용:")
print(yaml_content)

YAML 파일이 생성되었습니다: /content/drive/MyDrive/nipa_google/data/test/yolov8/dataset.yaml
내용:

path: /content/drive/MyDrive/nipa_google/data/test/yolov8  # 데이터셋 루트 경로
train:  # 더미 경로
val:   # 더미 경로
test: images  # 테스트 이미지 상대 경로
nc: 3  # 클래스 수
names: ['0', '1', '2']  # 클래스 이름



# 3. 성능 평가

In [ ]:
model = YOLO(model_path)

# 평가 실행
metrics = model.val(
    data=yaml_path,
    save_dir="/content/drive/MyDrive/nipa_google/model",  # 사용자 정의 경로
    imgsz=640,  # 입력 이미지 크기 (모델 학습 시 사용한 크기에 맞춰야 함)
    conf=0.001,  # 낮은 신뢰도 임계값 설정 (모든 결과 확인)
    iou=0.5,  # IoU 임계값
    split='test',  # 테스트 데이터셋 사용
    save=True  # 예측 결과 저장
)


# 평가 결과 출력
print("\n--- Performance Metrics ---")
print(f"mAP@0.5: {metrics.box.map50:.4f}")          # mAP@0.5
print(f"mAP@0.5:0.95: {metrics.box.map:.4f}")       # mAP@0.5:0.95
print(f"Precision: {metrics.box.p.mean():.4f}")    # Precision
print(f"Recall: {metrics.box.r.mean():.4f}")          # Recall

Ultralytics 8.3.39 🚀 Python-3.10.12 torch-2.5.1+cu121 CPU (Intel Xeon 2.20GHz)
Model summary (fused): 168 layers, 3,006,233 parameters, 0 gradients, 8.1 GFLOPs


val: Scanning /content/drive/MyDrive/nipa_google/data/test/yolov8/labels.cache... 236 images, 8 backgrounds, 0 corrupt: 100%|██████████| 244/244 [00:00<?, ?it/s]
                 Class     Images  Instances      Box(P          R      mAP50  mAP50-95): 100%|██████████| 16/16 [01:20<00:00,  5.05s/it]


                   all        244        383        0.8      0.539      0.575      0.326
            motorcycle         84        101      0.781      0.495      0.536      0.293
               bicycle         96        146      0.728      0.582      0.594      0.318
             kickboard        119        136      0.891       0.54      0.597      0.367
Speed: 8.6ms preprocess, 303.9ms inference, 0.0ms loss, 1.1ms postprocess per image
Results saved to runs/detect/val17

--- Performance Metrics ---
mAP@0.5: 0.5755
mAP@0.5:0.95: 0.3257
Precision: 0.7998
Recall: 0.5390

Evaluation results and visualizations are saved in: /content/drive/MyDrive/nipa_google/data/test/yolov8/test


# 4. 추론 속도 측정

In [ ]:
# 테스트 이미지 리스트 로드
image_files = [os.path.join(images_dir, img) for img in os.listdir(images_dir) if img.endswith(('.png', '.jpg', '.jpeg'))]

# 각 이미지에 대해 추론 속도 측정
inference_times = []

print("\n--- Inference Times per Image ---")
for image_path in image_files:
    start_time = time.time()  # 추론 시작 시간
    results = model.predict(image_path, imgsz=640, conf=0.001, iou=0.5)  # 추론 실행
    end_time = time.time()  # 추론 종료 시간

    inference_time = end_time - start_time
    inference_times.append(inference_time)

    # print(f"Image: {image_path}, Inference Time: {inference_time:.4f} seconds")

# 평균 추론 시간 계산
average_inference_time = sum(inference_times) / len(inference_times)
print(f"\n--- Average Inference Time ---")
print(f"Average Time per Image: {average_inference_time:.4f} seconds")